# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [16]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
import json
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Import the data to be analyzed:

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [3]:
aqdet.LoadDetectors(['taggerNewer.json'])

taggerNewer.json


We set up histograms for each of the channels, and then fill them with tagger times from our data:

In [4]:
taggerChannels = aqdet.Channels['Tagger']     # number of channels in the tagger (368)

histos = [None]*taggerChannels                # set up an array of 368 histograms to fill      

for i in range(taggerChannels):               # for each tagger channel
    histos[i] = Hist(1300,-500,800)           # set up a histogram for that tagger channel

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)              # calibrate the data
    taggerTimes = aqdet.Arrays['Tagger']['Time']     # get the tagger times
    
    for dat in taggerTimes:                          # for each data point
        for time in dat['Time']:                     # for each tagger time
            for i in range(taggerChannels):          # for each channel
                if (dat['channel']==i):              # if the channel of that data point is the current channel
                    histos[i].Fill(time)             # then put that time in the histogram
                    
    if(aq.eventNo%5000==0):                          # print a processing statement every 5000 events
        print("Events Processed: ",aq.eventNo)
    
aq.runFunction(plotCalTagger,0,500000)               # run this process over 500000 events

('Events Processed: ', 5000)
('Events Processed: ', 10000)
('Events Processed: ', 15000)
('Events Processed: ', 20000)
('Events Processed: ', 25000)
('Events Processed: ', 30000)
('Events Processed: ', 35000)
('Events Processed: ', 40000)
('Events Processed: ', 45000)
('Events Processed: ', 50000)
('Events Processed: ', 55000)
('Events Processed: ', 60000)
('Events Processed: ', 65000)
('Events Processed: ', 70000)
('Events Processed: ', 75000)
('Events Processed: ', 80000)
('Events Processed: ', 85000)
('Events Processed: ', 90000)
('Events Processed: ', 95000)
('Events Processed: ', 100000)
('Events Processed: ', 105000)
('Events Processed: ', 110000)
('Events Processed: ', 115000)
('Events Processed: ', 120000)
('Events Processed: ', 125000)
('Events Processed: ', 130000)
('Events Processed: ', 135000)
('Events Processed: ', 140000)
('Events Processed: ', 145000)
('Events Processed: ', 150000)
('Events Processed: ', 155000)
('Events Processed: ', 160000)
('Events Processed: ', 16500

Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them:

In [128]:
ROOT.enableJSVis()

tagger_times = ROOT.TFile("tagger_times.root","RECREATE")     # create a root file to put all the histograms into

myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")                   # make a gaussian fit with an offset/background
myFit.SetParNames("constant","mean","stdev","background")     # label the parameters

peak = [None]*taggerChannels
sigma = [None]*taggerChannels
bgLevel = [None]*taggerChannels

pk = 140
sig = 1.1
bg = 100

for i in range(taggerChannels):                                        
    myFit.SetParameter(1,pk)                 
    myFit.SetParameter(2,sig)                
    myFit.SetParameter(3,bg)                
    
    histos[i].SetTitle("Channel "+str(i))      
    histos[i].Draw()                
    
    if(histos[i].GetEntries()>10):                                     
        histos[i].Fit('myFit','SQ','',-300,600)                           
        peak[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(1)
        sigma[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(2)
        bgLevel[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(3) 
        
        k=1
        while(ROOT.gMinuit.fCstatu!="CONVERGED " or peak[i] > 160 or peak[i] < 120 or sigma[i]<-2 or sigma[i]>4):                                     
            myFit.SetParameter(1,140-k)                 
            myFit.SetParameter(2,1)                   
            myFit.SetParameter(3,80-k)
            
            peak[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(1)
            sigma[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(2)
            bgLevel[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(3) 
            
            k+=1
            #print("loop",i,k,peak[i],sigma[i],bgLevel[i])
            
        pk = peak[i]
        sig = sigma[i]
        bg = bgLevel[i]
  #      
        if (peak[i]>180 or peak[i]<100):
            print("Check channel "+ str(i)+" for peak")
            print(i,k,pk,sig,bg)
       
        if (sigma[i]<-2):
            print("Check channel "+ str(i)+" for sigma")
            print(i,k,pk,sig,bg)
   #         
    else:                                            
        print("Histogram "+str(i)+" is empty (has "+str(histos[i].GetEntries())+" entries).")   
     
    histos[i].Write("tagger_times")    

Histogram 216 is empty (has 0.0 entries).
Histogram 273 is empty (has 1.0 entries).
Histogram 327 is empty (has 1.0 entries).


Finding how much each peak is off from zero, and setting it back to zero:

In [55]:
bins = 1300
ROOT.enableJSVis()
tagger_calib = ROOT.TFile("tagger_calib.root","RECREATE")

histos_calib = [None]*taggerChannels
for i in range(taggerChannels):
    histos_calib[i] = Hist(bins,-500,800)

for i in range(taggerChannels):
    print(i, peak[i])
    if (peak[i]!=None):
        histos_calib[i] = Hist(bins,int(-500-peak[i]),int(800-peak[i]))
        for j in range(bins):
            histos_calib[i].SetBinContent(j,histos[i].GetBinContent(j))
     
    histos_calib[i].SetTitle("Channel "+str(i))      
    histos_calib[i].Draw() 
    
    histos_calib[i].Write("tagger_calib")  

(0, 141.35560206616336)
(1, 142.34234382101374)
(2, 142.04737942716037)
(3, 141.13014167529874)
(4, 143.9712383600463)
(5, 142.07514793480138)
(6, 142.28037937316245)
(7, 141.94766430684325)
(8, 141.4015973974516)
(9, 142.59829127633077)
(10, 141.5765482119733)
(11, 140.96162242169027)
(12, 141.19443437104218)
(13, 141.3893402326737)
(14, 142.0777289011689)
(15, 142.26574410774904)
(16, 143.6001557985984)
(17, 144.4307032957878)
(18, 143.28335249970635)
(19, 143.13576442703018)
(20, 143.1396795056008)
(21, 143.61878434679946)
(22, 143.90687599386914)
(23, 143.88424028007836)
(24, 140.854141538865)
(25, 145.3690070151206)
(26, 140.48487504219398)
(27, 140.47283787283374)
(28, 140.86532795068922)
(29, 141.0151873016842)
(30, 141.73565588681217)
(31, 142.91309572256645)
(32, 140.25139836829558)
(33, 141.1606946421971)
(34, 139.75299225191773)
(35, 139.8839830982344)
(36, 139.9725295182527)
(37, 140.46380008893522)
(38, 143.86671167884143)
(39, 140.61188790566635)
(40, 141.38555583914172)


(340, 143.80684235011222)
(341, 144.60321621289165)
(342, 143.99835070810173)
(343, 144.03590939492418)
(344, 142.88695622918337)
(345, 144.42091962855426)
(346, 144.05846678916402)
(347, 143.94397764561137)
(348, 145.45323938105963)
(349, 146.1991867196631)
(350, 145.9422812982191)
(351, 145.9253228690477)
(352, 144.93021150610866)
(353, 145.93096765116204)
(354, 144.63765276111118)
(355, 145.50179830479888)
(356, 144.76753609733618)
(357, 145.92671637385115)
(358, 145.31145294107586)
(359, 145.59812719681577)
(360, 143.15089483369093)
(361, 143.9998101304656)
(362, 143.1957378443753)
(363, 143.0945965916003)
(364, 142.74862772504895)
(365, 143.76443508386575)
(366, 142.78667445064374)
(367, 143.25313693345737)


Next, we must import the json file, change the channel offset values, and then save the array again as a new json file. Note that the empty histograms write a value of 'None' to the json file for their offsets.

In [125]:
calibData = json.load(open('taggerNewer.json'), object_pairs_hook=OrderedDict)     # load the tagger calibration data and keep its original ordering

i = 0                                   # starting at channel 0
for chan in calibData['channels']:      # for each of the tagger channels
    chan['offset'][0] = peak[i]         # change the offset value to the one found in our earlier fit for that channel
    i+=1                                # move on to the next channel

json.dump(calibData, open('taggerWithOffsets.json', 'w'), indent = 2)              # write the changed data to a new json file